In [10]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("="*80)
print("STEP 1: DATA PREPARATION")
print("="*80)

STEP 1: DATA PREPARATION


In [9]:
print("\n1.1 Loading data files...")

# Loading the three CSV files
demand = pd.read_csv('demand.csv')
plants = pd.read_csv('plants.csv')
costs = pd.read_csv('generation_costs.csv')

print(f" Demand data loaded: {demand.shape}")
print(f" Plants data loaded: {plants.shape}")
print(f" Costs data loaded: {costs.shape}")


1.1 Loading data files...
 Demand data loaded: (500, 15)
 Plants data loaded: (64, 21)
 Costs data loaded: (32000, 3)


In [4]:
# 1.2 HANDLING MISSING VALUES
print("\n1.2 Handling missing values...")

# Check how many missing values we have
print("\nMissing values BEFORE handling:")
print(f"  Demand: {demand.isnull().sum().sum()} missing")
print(f"  Plants: {plants.isnull().sum().sum()} missing")
print(f"  Costs: {costs.isnull().sum().sum()} missing")

# Create copies of the data
demand_clean = demand.copy()
plants_clean = plants.copy()
costs_clean = costs.copy()

# Identify column types
# Demand columns
demand_numerical = [col for col in demand_clean.columns 
                    if col.startswith('DF') and col not in ['DF_region', 'DF_daytype']]
demand_categorical = ['DF_region', 'DF_daytype']

# Plant columns
plant_numerical = [col for col in plants_clean.columns if col.startswith('PF')]
plant_categorical = ['Plant Type', 'Region']

# Fill missing values in DEMAND
for col in demand_numerical:
    if demand_clean[col].isnull().sum() > 0:
        median = demand_clean[col].median()
        demand_clean[col].fillna(median, inplace=True)

# For categories: use mode (most common)
for col in demand_categorical:
    if demand_clean[col].isnull().sum() > 0:
        mode = demand_clean[col].mode()[0]
        demand_clean[col].fillna(mode, inplace=True)

# Fill missing values in PLANTS
for col in plant_numerical:
    if plants_clean[col].isnull().sum() > 0:
        median = plants_clean[col].median()
        plants_clean[col].fillna(median, inplace=True)

# For categories: use 'Unknown'
for col in plant_categorical:
    if plants_clean[col].isnull().sum() > 0:
        plants_clean[col].fillna('Unknown', inplace=True)

# Fill missing values in COSTS
if costs_clean['Cost_USD_per_MWh'].isnull().sum() > 0:
    costs_clean['Cost_USD_per_MWh'] = costs_clean.groupby('Plant ID')['Cost_USD_per_MWh'].transform(
        lambda x: x.fillna(x.median())
    )
    if costs_clean['Cost_USD_per_MWh'].isnull().sum() > 0:
        costs_clean['Cost_USD_per_MWh'].fillna(costs_clean['Cost_USD_per_MWh'].median(), inplace=True)

print("\nMissing values AFTER handling:")
print(f"  Demand: {demand_clean.isnull().sum().sum()} missing")
print(f"  Plants: {plants_clean.isnull().sum().sum()} missing")
print(f"  Costs: {costs_clean.isnull().sum().sum()} missing")


1.2 Handling missing values...

Missing values BEFORE handling:
  Demand: 313 missing
  Plants: 19 missing
  Costs: 96 missing

Missing values AFTER handling:
  Demand: 0 missing
  Plants: 0 missing
  Costs: 0 missing


In [5]:
# 1.3 FEATURE SELECTION
print("\n1.3 Selecting important features...")

# Step 1: Remove low variance features
demand_variances = demand_clean[demand_numerical].var()
low_variance = demand_variances[demand_variances < 0.01].index.tolist()

# Step 2: Remove highly correlated features
demand_corr = demand_clean[demand_numerical].corr()
high_corr_pairs = []

for i in range(len(demand_corr.columns)):
    for j in range(i+1, len(demand_corr.columns)):
        if abs(demand_corr.iloc[i, j]) > 0.9:  # If correlation > 0.9
            col1 = demand_corr.columns[i]
            col2 = demand_corr.columns[j]
            high_corr_pairs.append((col1, col2))

# Remove features
to_remove = set(low_variance)
for feat1, feat2 in high_corr_pairs:
    # Keep the one with higher variance
    if demand_variances[feat1] < demand_variances[feat2]:
        to_remove.add(feat1)
    else:
        to_remove.add(feat2)

selected_demand_features = [col for col in demand_numerical if col not in to_remove]

print(f"\nDemand features:")
print(f"  Original: {len(demand_numerical)}")
print(f"  Selected: {len(selected_demand_features)}")
print(f"  Features: {selected_demand_features}")

# PLANT FEATURES
plant_variances = plants_clean[plant_numerical].var()
low_var_plants = plant_variances[plant_variances < 0.01].index.tolist()

plant_corr = plants_clean[plant_numerical].corr()
plant_high_corr = []

for i in range(len(plant_corr.columns)):
    for j in range(i+1, len(plant_corr.columns)):
        if abs(plant_corr.iloc[i, j]) > 0.9:
            col1 = plant_corr.columns[i]
            col2 = plant_corr.columns[j]
            plant_high_corr.append((col1, col2))

plant_to_remove = set(low_var_plants)
for feat1, feat2 in plant_high_corr:
    if plant_variances[feat1] < plant_variances[feat2]:
        plant_to_remove.add(feat1)
    else:
        plant_to_remove.add(feat2)

selected_plant_features = [col for col in plant_numerical if col not in plant_to_remove]

print(f"\nPlant features:")
print(f"  Original: {len(plant_numerical)}")
print(f"  Selected: {len(selected_plant_features)}")
print(f"  Features: {selected_plant_features}")


1.3 Selecting important features...

Demand features:
  Original: 12
  Selected: 12
  Features: ['DF1', 'DF2', 'DF3', 'DF4', 'DF5', 'DF6', 'DF7', 'DF8', 'DF9', 'DF10', 'DF11', 'DF12']

Plant features:
  Original: 18
  Selected: 18
  Features: ['PF1', 'PF2', 'PF3', 'PF4', 'PF5', 'PF6', 'PF7', 'PF8', 'PF9', 'PF10', 'PF11', 'PF12', 'PF13', 'PF14', 'PF15', 'PF16', 'PF17', 'PF18']


In [6]:
# 1.4 FEATURE SCALING
print("\n1.4 Scaling features to standard range...")

# Create scalers
scaler_demand = StandardScaler()
scaler_plant = StandardScaler()

# Scale demand features (mean=0, std=1)
demand_clean[selected_demand_features] = scaler_demand.fit_transform(
    demand_clean[selected_demand_features]
)

# Scale plant features (mean=0, std=1)
plants_clean[selected_plant_features] = scaler_plant.fit_transform(
    plants_clean[selected_plant_features]
)

print(f" Demand features scaled (mean ≈ 0, std ≈ 1)")
print(f" Plant features scaled (mean ≈ 0, std ≈ 1)")


1.4 Scaling features to standard range...
 Demand features scaled (mean ≈ 0, std ≈ 1)
 Plant features scaled (mean ≈ 0, std ≈ 1)


In [7]:
# 1.5 REMOVING WORST-PERFORMING PLANTS
print("\n1.5 Removing worst-performing plants...")

# Calculate average cost for each plant
plant_performance = costs_clean.groupby('Plant ID')['Cost_USD_per_MWh'].mean().reset_index()
plant_performance.columns = ['Plant_ID', 'Mean_Cost']
plant_performance = plant_performance.sort_values('Mean_Cost')

print(f"\nBest 5 plants (lowest cost):")
print(plant_performance.head(5).to_string(index=False))

print(f"\nWorst 5 plants (highest cost):")
print(plant_performance.tail(5).to_string(index=False))

# Remove worst 25% (plants with highest average cost)
threshold = plant_performance['Mean_Cost'].quantile(0.75)
worst_plants = plant_performance[plant_performance['Mean_Cost'] > threshold]['Plant_ID'].tolist()

print(f"\nRemoving {len(worst_plants)} worst plants (top 25% by cost)")
print(f"Cost threshold: ${threshold:.2f}")

# Remove from data
plants_clean = plants_clean[~plants_clean['Plant ID'].isin(worst_plants)]
costs_clean = costs_clean[~costs_clean['Plant ID'].isin(worst_plants)]

print(f"\nPlants remaining: {len(plants_clean)} (from {len(plants)})")
print(f"Cost records remaining: {len(costs_clean)} (from {len(costs)})")


1.5 Removing worst-performing plants...

Best 5 plants (lowest cost):
Plant_ID  Mean_Cost
     P28  43.747682
     P42  44.545986
     P15  44.799738
     P26  47.331016
     P33  49.413156

Worst 5 plants (highest cost):
Plant_ID  Mean_Cost
     P46  72.360404
     P58  73.506250
      P4  74.048046
      P5  76.667750
      P2  76.804356

Removing 16 worst plants (top 25% by cost)
Cost threshold: $64.15

Plants remaining: 48 (from 64)
Cost records remaining: 24000 (from 32000)


In [8]:
# 1.6 SAVING CLEANED DATA
print("\n1.6 Saving cleaned data files...")

# Prepare final datasets
demand_final = demand_clean[['Demand ID', 'DF_region', 'DF_daytype'] + selected_demand_features]
plants_final = plants_clean[['Plant ID', 'Plant Type', 'Region'] + selected_plant_features]
costs_final = costs_clean

# Save to CSV
demand_final.to_csv('demand_clean.csv', index=False)
plants_final.to_csv('plants_clean.csv', index=False)
costs_final.to_csv('costs_clean.csv', index=False)

print(f" Saved demand_clean.csv ({demand_final.shape})")
print(f" Saved plants_clean.csv ({plants_final.shape})")
print(f" Saved costs_clean.csv ({costs_final.shape})")

# Save feature information
with open('feature_info.txt', 'w') as f:
    f.write("SELECTED FEATURES\n")
    f.write("="*80 + "\n\n")
    f.write(f"Demand features ({len(selected_demand_features)}):\n")
    f.write(str(selected_demand_features) + "\n\n")
    f.write(f"Plant features ({len(selected_plant_features)}):\n")
    f.write(str(selected_plant_features) + "\n\n")
    f.write(f"Plants retained: {len(plants_final)}\n")
    f.write(f"Plants removed: {len(worst_plants)}\n")

print(f" Saved feature_info.txt")



1.6 Saving cleaned data files...
 Saved demand_clean.csv ((500, 15))
 Saved plants_clean.csv ((48, 21))
 Saved costs_clean.csv ((24000, 3))
 Saved feature_info.txt
